## 0. 필요 라이브러리 import

In [1]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery

## 1. Pandas 라이브러리를 활용하여 BigQuery 다루기

#### 1-1) GCP 권한 인증

In [2]:
JSON_KEY_PATH = "sprintda05-soomin-389e878cb2dc.json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

### 1-2) 고정값 정의

In [3]:
project_id = 'sprintda05-soomin'
dataset = 'sprint_pokemon'
table = 'pokemon'
location = 'asia-northeast3'
query = f"SELECT * FROM {project_id}.{dataset}.{table}"

#### 1-3) 데이터 읽기

In [6]:
pd.read_gbq(
    query = query,
    project_id=project_id,
    credentials=credentials,
    location=location
)

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
0,1,이상해씨,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,이상해풀,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,이상해꽃,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,4,파이리,Charmander,Fire,None,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,None,405,58,64,58,80,65,80,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,247,데기라스,Pupitar,Rock,Ground,410,70,84,70,65,70,51,2,False
247,248,마기라스,Tyranitar,Rock,Dark,600,100,134,110,95,100,61,2,False
248,249,루기아,Lugia,Psychic,Flying,680,106,90,130,90,154,110,2,True
249,250,칠색조,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True


#### 1-4) 데이터 쓰기

In [8]:
point_his = pd.read_parquet('dataset/point_his.parquet')
point_his.head()

,idx,proc_ym,proc_ymd,point
0,96465,202306,20230624,1000
1,96465,202306,20230624,500
2,87940,202304,20230405,2000
3,87940,202304,20230405,3500
4,87940,202304,20230405,4000


In [9]:
point_his.to_gbq(
    destination_table='bigquery.point_his',
    project_id=project_id,
    if_exists='replace',
    location=location,
    credentials=credentials
)

## 2. google-cloud-bigquery 라이브러리를 활용하여 BigQuery 다루기

#### 2-1) GCP 권한 인증 및 Client 생성
- Client - BigQuery에 사용자가 내린 명령을 API로 전달하고 결과를 가져오는 역할

In [11]:
# 상대경로로 사용
JSON_KEY_PATH = "sprintda05-soomin-389e878cb2dc.json"
credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

project_id = 'sprintda05-soomin'
dataset = 'sprint_pokemon'
table = 'pokemon'
location = 'asia-northeast3'

In [10]:
client = bigquery.Client(
    project=project_id,
    location=location,
    credentials=credentials   
)


#### 2-2) Dataset 관련 실습

In [13]:
## dataset 생성
dataset_name = 'python_database'

dataset_obj = bigquery.Dataset(f"{project_id}.{dataset_name}")

client.create_dataset(
    dataset=dataset_obj,
    exists_ok=True,
)

Dataset(DatasetReference('sprintda05-soomin', 'python_database'))

In [14]:
## dataset 목록 조회
for dataset in client.list_datasets():
    print(dataset.dataset_id)

bigquery
python_database
sprint_pokemon


In [ ]:
## 데이터셋 삭제
client.delete_dataset(
    dataset=f'{project_id}.{dataset_name}',
    not_found_ok=True
)

In [16]:
## dataset 목록 조회
for dataset in client.list_datasets():
    print(dataset.dataset_id)

bigquery
sprint_pokemon


In [20]:
## 해당 객체가 사용할 수 있는 메소드를 모두 다 출력!
dir(client)

['SCOPE',
 '_SET_PROJECT',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_api',
 '_client_cert_source',
 '_connection',
 '_credentials',
 '_dataset_from_arg',
 '_default_load_job_config',
 '_default_query_job_config',
 '_determine_default',
 '_do_multipart_upload',
 '_do_resumable_upload',
 '_ensure_bqstorage_client',
 '_get_query_results',
 '_http',
 '_http_internal',
 '_initiate_resumable_upload',
 '_list_rows_from_query_results',
 '_location',
 '_schema_from_json_file_object',
 '_schema_to_json_file_object',
 'cancel_job',
 'close',
 'copy_table',
 'create_dataset',
 'create_job',
 'create_routi

#### 2-3) Table 관련 실습

In [21]:
## 테이블 목록 조회
for table in client.list_tables(dataset='sprint_pokemon'):
    print(table.table_id)

member
pokemon


In [22]:
## 테이블 조회
'''
client.query(
    query = query,
    project = project_id,
    location = location
    )

이지만 앞에서 이미 선언 하였으므로 생략해도 작동
'''

table_obj = client.query(
    query='SELECT * FROM sprint_pokemon.pokemon'
)

In [ ]:
dir(table_obj)

In [26]:
table_obj.to_dataframe().head()

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
0,1,이상해씨,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,이상해풀,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,이상해꽃,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,4,파이리,Charmander,Fire,None,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,None,405,58,64,58,80,65,80,1,False


In [27]:
## 테이블을 다른 dataset으로 복제
# sprint_pokemon.member -> bigquery.member

client.copy_table(
    sources='sprint_pokemon.member',
    destination='bigquery.member'
)


CopyJob<project=sprintda05-soomin, location=asia-northeast3, id=729432ad-ca30-4b3d-a8d7-207e52f27e53>

In [28]:
## dataframe -> Bigquery Table
regdate = pd.read_parquet('./dataset/regdate.parquet')
regdate.head()

,idx,regdate
0,1,20221206
1,2,20221206
2,3,20221206
3,4,20221206
4,5,20221206


In [ ]:
# 데이터프레임을 빅쿼리 테이블로 생성
client.load_table_from_dataframe(
    dataframe=regdate,
    destination='sprint_pokemon.regdate'
)

LoadJob<project=sprintda05-soomin, location=asia-northeast3, id=0fdf87c3-971e-4047-8516-31fcb3a57981>

In [30]:
## 테이블 삭제
client.delete_table(
    table='sprint_pokemon.regdate',
    not_found_ok=True
)

#### 2-4) 실습 미션 - 1

dataset/ 디렉토리 아래의 5개 parquet 파일을 대상으로 작업.

- 데이터셋 ID : education
- 테이블명 : 파일명과 동일
- 요구사항
    1. bigquery client를 활용하여 작업해주세요
    2. 'education' 이라는 이름의 새로운 데이터셋을 생성해주세요.
    3. 해당 데이터셋 내부에 5개의 테이블을 저장해주세요.
    4. 'python_dataset' 데이터셋으로 member 제외 나머지 테이블 복사.
    
  권장 사항!
  - 3번, 4번을 for문 써서 어떻게 구현할 수 있을지 고민!

In [36]:
import os

# 1
JSON_KEY_PATH = 'sprintda05-soomin-389e878cb2dc.json'
credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

project_id = 'sprintda05-soomin'
location = 'asia-northeast3'

client = bigquery.Client(
    project=project_id,
    location=location,
    credentials=credentials   
)

# 2
client.create_dataset(
    dataset=f'{project_id}.education',
    exists_ok=True
)

# 3
parquet_files = [f for f in os.listdir('dataset') if f.endswith('.parquet')]
dataframes = [pd.read_parquet(os.path.join('dataset', f)) for f in parquet_files]
for i, file in enumerate(parquet_files):
    name = os.path.splitext(file)[0]
    
    client.load_table_from_dataframe(
        dataframe=dataframes[i],
        destination=f'education.{name}'
)

# 4
client.create_dataset(
    dataset=f'{project_id}.python_dataset',
    exists_ok=True
)

for table in client.list_tables(dataset='education'):
    name = table.table_id
    if name != 'member':
        client.copy_table(
            sources=f'education.{name}',
            destination=f'python_dataset.{name}'
)


#### 2-5) BigQuery 테이블 → GCS의 파일로 저장

In [40]:
## CSV 파일로 클라우드 버킷에 저장
bucket_name = 'sprintda05-soomin-bucket'

client.extract_table(
    source=f'education.point_his',
    destination_uris=[f'gs://{bucket_name}/education/point_his.csv']
)


ExtractJob<project=sprintda05-soomin, location=asia-northeast3, id=da5adacf-2df9-4d67-b36d-297b2c1bdfdf>

In [41]:
## 압축 파일로 저장
job_config = bigquery.job.ExtractJobConfig()
job_config.compression = bigquery.Compression.GZIP

client.extract_table(
    source=f'education.point_his',
    destination_uris=[f'gs://{bucket_name}/education/point_his.csv.gz'],
    job_config= job_config
)

ExtractJob<project=sprintda05-soomin, location=asia-northeast3, id=bcd87553-2a21-413d-9aba-c26999da8be3>

#### 2-6) GCS의 파일 → BigQuery 테이블로 Load

In [52]:
## CSV 파일 
job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,        # csv는 첫 행을 skip해서 컬럼으로 써야 함
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV
    )

client.load_table_from_uri(
    source_uris=[f'gs://{bucket_name}/education/point_his.csv'],
    destination=f'{project_id}.education.point_his',
    job_config=job_config
)

LoadJob<project=sprintda05-soomin, location=asia-northeast3, id=de906856-7de3-42e6-8849-c2ae42eb4096>

In [53]:
## parquet 파일
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.PARQUET
    )

client.load_table_from_uri(
    source_uris="gs://sprintda05-soomin-bucket/BigQuery/item_his.parquet",
    destination="education.item_his",
    job_config=job_config
)


LoadJob<project=sprintda05-soomin, location=asia-northeast3, id=91d47056-b4a2-453f-940d-6e9fe6f9e83e>